# Сентимент-анализ текстов на русском языке

In [1]:
import os

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import nltk
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import TfidfVectorizer
#TODO мб попробовать word2vec/fasttext

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, confusion_matrix

Запуск скрипта для загрузки тренировочных и тестовых данных (можно не комментировать, в случае 
если данные уже загружены - это учитывается) и загрузка данных в датафреймы

In [2]:
current_dir = os.getcwd()
project_dir = os.path.abspath(os.path.join(current_dir, '..'))

data_load_script_path = os.path.join(project_dir, "scripts\\data_load.py")
!python {data_load_script_path}

data_dir = os.path.join(project_dir, 'data')

train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'test.csv'))

Посмотрим на данные

In [3]:
train.head(5)

,Unnamed: 0,text,sentiment
0,21098,".с.,и спросил его: о Посланник Аллаха!Ты пори...",1
1,21099,Роднее всех родных Попала я в ГКБ №8 еще в дек...,1
2,21100,Непорядочное отношение к своим работникам Рабо...,2
3,21101,"). Отсутствуют нормативы, Госты и прочее, что ...",1
4,21102,У меня машина в руках 5 лет и это п...,1


0 - нейтральный, 1 - позитивный, 2 - негативный.  
Можно отбросить признак Unnamed: 0

In [4]:
train.drop(columns=["Unnamed: 0"], inplace=True)
test.drop(columns=["Unnamed: 0"], inplace=True)

In [5]:
train.describe()

,sentiment
count,189891.00000
mean,1.00248
std,0.72250
min,0.00000
25%,0.00000
50%,1.00000
75%,2.00000
max,2.00000


In [6]:
train.nunique()

text         181103
sentiment         3
dtype: int64

In [7]:
train.isnull().sum()

text         0
sentiment    0
dtype: int64

In [8]:
train['sentiment'].value_counts()

sentiment
1    90766
2    49798
0    49327
Name: count, dtype: int64

Видим - с данными все в порядке, пропусков нет. Отметим несбалансированность положительных 
примеров, возможно пригодится.

## Предобработка

Функции предобработки для текста и датасета

In [9]:
morph = MorphAnalyzer()
# nltk.download('stopwords') # Не надо выполнять если пакет уже загружен
stop_words = set(stopwords.words('russian'))


def preprocess_text(text):
    """
    Предобработка текста для сентимент-анализа: удаление стоп-слов и лишних символов, 
    лемматизация, токенизация
    :param text: Текст, который надо обработать
    :return: Обработанные токены
    """
    
    text = text.lower()
    text = re.sub(r'[^а-яёЁ?!-+0-9\s]', '', text)
    text = text.replace('!', ' ! ').replace('!?', ' !? ')
    text = re.sub(r'[()\[\]{}]', '', text)

    tokens = text.split()
    tokens = [morph.parse(token)[0].normal_form for token in tokens if token not in stop_words]
    return tokens


def preprocess(dataset):
    """
    Создание дополнительного признака в датасете - результат применения preprocess_text к тексту.
    :param dataset: Датасет, который надо обработать
    :return: None
    """

    tqdm.pandas(desc="Обработка датасета")

    dataset["prep_text"] = dataset["text"].progress_apply(preprocess_text)
    
    return None

Применение функции предобработки к датасетам и их сохранение в зависимости от того, было ли это 
сделано ранее

In [10]:
prep_train_file = os.path.join(data_dir, 'prep_train.csv')
prep_test_file = os.path.join(data_dir, 'prep_test.csv')

if not os.path.exists(prep_train_file):
    print("Обработка тренировочного датасета")
    preprocess(train)
    train.to_csv(prep_train_file, index=False)  # Сохранение обработанных данных
    print(f"Обработанные тренировочные данные сохранены")


if not os.path.exists(prep_test_file):
    print("Обработка тестового датасета")
    preprocess(test)
    test.to_csv(prep_test_file, index=False)  # Сохранение обработанных данных
    print(f"Обработанные тестовые данные сохранены")

prep_train = pd.read_csv(os.path.join(data_dir, 'prep_train.csv'))
prep_test = pd.read_csv(os.path.join(data_dir, 'prep_test.csv'))

## Обучение различных моделей